In [ ]:
!pip install matgraphdb
!pip install ipykernel

# Introduction to MatGraphDB

This notebook demonstrates the basic usage of MatGraphDB, a graph database designed for materials science data. MatGraphDB allows you to:

1. Store and query materials data
2. Create relationships between different types of materials data
3. Generate and manage derived data through generators

Let's go through some basic examples.

In [1]:
import os
import shutil
import pandas as pd

## 1. Creating a MatGraphDB Instance

The main class in MatGraphDB is the `MatGraphDB` class, which can be directly imported from the `matgraphdb` package. This class is used to create a new MatGraphDB instance. We'll store our data in a temporary directory.

In [2]:
from matgraphdb import MatGraphDB

# Create a temporary directory for our database

storage_path = "MatGraphDB"
if os.path.exists(storage_path):
    shutil.rmtree(storage_path)

os.makedirs(storage_path, exist_ok=True)


# Initialize MatGraphDB
mgdb = MatGraphDB(storage_path=storage_path)

Let's take a look at the database summary

In [3]:
print(mgdb.summary())

GRAPH DATABASE SUMMARY
Name: MatGraphDB
Storage path: MatGraphDB
└── Repository structure:
    ├── nodes/                 (MatGraphDB\nodes)
    ├── edges/                 (MatGraphDB\edges)
    ├── edge_generators/       (MatGraphDB\edge_generators)
    ├── node_generators/       (MatGraphDB\node_generators)
    └── graph/                 (MatGraphDB\graph)

############################################################
NODE DETAILS
############################################################
Total node types: 1
------------------------------------------------------------
• Node type: materials
  - Number of nodes: 0
  - Number of features: 1
  - db_path: MatGraphDB\nodes\materials
------------------------------------------------------------

############################################################
EDGE DETAILS
############################################################
Total edge types: 0
------------------------------------------------------------

###############################

You'll notice that a "materials" node store has been automatically created. This is because the `MatGraphDB` class automatically creates a node store for the "materials" node type.

## 2. Adding Materials

We can create new materials in our "materials" store by calling `mgdb.create_material(...)`. This is a wrapper around the `create_material` method of the `MaterialNodes` class, which mean the arugments are the same.

Materials can be created in `MatGraphDB` in two primary ways through the `create_materials` method:

1. **Using a `pymatgen` `Structure` object**:  
   This is the most direct way when you already have a `Structure` object that defines the material's atomic arrangement.

2. **Providing atomic coordinates, species, and lattice manually**:  
   This is useful when you have the raw data and want to construct the material without a `Structure` object.  

Additionally, you can include custom properties in the form of a nested dictionary to enrich the material data.


#### Example 1: Using a `Structure` object

In [4]:
# Define a pymatgen Structure object
structure = Structure(
    lattice=[[0, 2.13, 2.13], [2.13, 0, 2.13], [2.13, 2.13, 0]],
    species=["Mg", "O"],
    coords=[[0, 0, 0], [0.5, 0.5, 0.5]],
)

material_data = {
    "structure": structure,
    "properties": {
        "material_id": "mp-1",
        "source": "example",
        "thermal_conductivity": {"value": 2.5, "unit": "W/mK"},
    },
}

# Add the material to the database.
mgdb.create_material(
    structure=material_data["structure"], properties=material_data["properties"]
)

material_nodes = mgdb.read_nodes("materials")
print(material_nodes.to_pandas())

2025-01-10 17:17:41 - matgraphdb.materials.nodes.materials - INFO - Adding a new material.
2025-01-10 17:17:41 - matgraphdb.materials.nodes.materials - INFO - Material added successfully.
  atomic_numbers                       cartesian_coords   density  \
0        [12, 8]  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843   

   density_atomic elements formula                         frac_coords  id  \
0        0.103481  [Mg, O]  Mg1 O1  [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]   0   

                                             lattice material_id  ...  \
0  [0.0, 2.13, 2.13, 2.13, 0.0, 2.13, 2.13, 2.13,...        mp-1  ...   

   structure.lattice.beta  structure.lattice.c structure.lattice.gamma  \
0                    60.0             3.012275                    60.0   

                            structure.lattice.matrix structure.lattice.pbc  \
0  [0.0, 2.13, 2.13, 2.13, 0.0, 2.13, 2.13, 2.13,...    [True, True, True]   

  structure.lattice.volume                                    s

#### Example 2: Using atomic coordinates, species, and lattice manually

In [5]:
# Define atomic data
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]
species = ["Mg", "O"]
lattice = [[0, 2.13, 2.13], [2.13, 0, 2.13], [2.13, 2.13, 0]]

# Add the material to the database
material_data = {
    "coords": coords,
    "species": species,
    "lattice": lattice,
    "properties": {
        "material_id": "mp-2",
        "source": "example_manual",
        "band_gap": {"value": 1.2, "unit": "eV"},
    },
}
result = mgdb.create_material(
    coords=material_data["coords"],
    species=material_data["species"],
    lattice=material_data["lattice"],
    properties=material_data["properties"],
)

material_nodes = mgdb.read_nodes("materials")
print(material_nodes.to_pandas())
print(material_nodes.shape)

2025-01-10 17:18:51 - matgraphdb.materials.core - INFO - Creating material.
2025-01-10 17:18:51 - matgraphdb.materials.nodes.materials - INFO - Adding a new material.
2025-01-10 17:18:51 - matgraphdb.materials.nodes.materials - INFO - Material added successfully.
  atomic_numbers band_gap.unit  band_gap.value  \
0        [12, 8]          None             NaN   
1        [12, 8]            eV             1.2   

                        cartesian_coords   density  density_atomic elements  \
0  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843        0.103481  [Mg, O]   
1  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843        0.103481  [Mg, O]   

  formula                         frac_coords  id  ... structure.lattice.beta  \
0  Mg1 O1  [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]   0  ...                   60.0   
1  Mg1 O1  [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]   1  ...                   60.0   

  structure.lattice.c  structure.lattice.gamma  \
0            3.012275                     60.0   
1  

### NodeStores and EdgeStores

Any nodes or edges in MatGraphDB are stored in **NodeStores** and **EdgeStores**, respectively. These can be accessed through the `node_stores` and `edge_stores` attributes of the `MatGraphDB` class. These stores extend the capabilities of the `ParquetDB` class, allowing users to leverage all the features available in `ParquetDB`. 

#### What is ParquetDB?

`ParquetDB` is a database framework built on top of **Apache Parquet** and **PyArrow**, optimized for handling large-scale data. Its core strength lies in its ability to efficiently store, query, and process vast amounts of data with minimal overhead. The foundation of `ParquetDB`—Apache Parquet—offers several advantages:

- **Columnar Format**: Parquet organizes data by columns instead of rows, making it particularly effective for analytical queries that only need specific columns. This format improves compression efficiency and reduces I/O overhead.  
- **Schema Embedding**: Each Parquet file includes an embedded schema, ensuring consistency and enabling seamless schema evolution.  
- **Predicate Pushdown**: Parquet's structure allows queries to read only the necessary data blocks and relevant columns, significantly improving query performance by reducing data load times.  
- **Efficient Encoding and Compression**: Parquet supports column-level encoding and compression, enhancing both storage efficiency and read performance.  
- **Metadata Support**: Parquet files store metadata at both the table and field levels, which facilitates efficient querying and rich data descriptions.  
- **Batch Processing**: Data in Parquet files is organized into column groups, making it ideal for batch operations and high-throughput workflows.

#### Why Use ParquetDB?

By leveraging the advantages of Parquet files, `ParquetDB` efficiently handles the serialization and deserialization of complex datasets. It provides scalable and fast access to data, seamlessly integrating into machine learning workflows and big data pipelines. This is especially beneficial for material science applications in MatGraphDB, where datasets often involve complex and interconnected relationships between nodes and edges.

#### Integration with MatGraphDB

When nodes are added to MatGraphDB, they are indexed, and the index is stored in the `id` column. This ensures fast lookups and efficient data management. Whether you are storing materials, elements, or defining relationships between them, NodeStores and EdgeStores inherit all the features of `ParquetDB`, making them robust, scalable, and performant.

For more details about `ParquetDB`, including its architecture and capabilities, refer to the [ParquetDB documentation](https://github.com/lllangWV/ParquetDB).


### Adding Multiple Materials at Once

If you have multiple materials, it is more efficient to add them in a bulk call. You can do this by using create_materials. Here's an example:

In [6]:
materials = [
    {
        "structure": structure,
        "properties": {"material_id": "mp-3", "density": 3.95},
    },
    {
        "coords": [[0, 0, 0], [0.5, 0.5, 0.5]],
        "species": ["Si", "O"],
        "lattice": [[0, 3.1, 3.1], [3.1, 0, 3.1], [3.1, 3.1, 0]],
        "properties": {"material_id": "mp-4", "band_gap": 1.8},
    },
]

mgdb.create_materials(materials)

material_nodes = mgdb.read_nodes("materials")
print(material_nodes.to_pandas())
print(material_nodes.shape)

2025-01-10 17:39:19 - matgraphdb.materials.core - INFO - Creating materials.
2025-01-10 17:39:19 - matgraphdb.materials.nodes.materials - INFO - Adding 2 materials to the database.
2025-01-10 17:39:19 - matgraphdb.utils.mp_utils - INFO - Passing the following arguments to the worker method
2025-01-10 17:39:21 - matgraphdb.materials.nodes.materials - ERROR - Error adding material: Unable to merge: Field atomic_numbers has incompatible types: list<element: int64> vs extension<arrow.fixed_shape_tensor[value_type=int64, shape=[2]]>
2025-01-10 17:39:21 - matgraphdb.materials.nodes.materials - INFO - All materials added successfully.
  atomic_numbers band_gap.unit  band_gap.value  \
0        [12, 8]          None             NaN   
1        [12, 8]            eV             1.2   

                        cartesian_coords   density  density_atomic elements  \
0  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843        0.103481  [Mg, O]   
1  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843     

## 3. Reading Materials

In the previous section, we added materials to the database and performed a basic read operation, but lets go into more details for the read operations.


To start, we can read all materials in the database with the `read_materials` method.

In [7]:
# Read all materials
materials = mgdb.read_materials()
print("All materials:")
print(materials.to_pandas())

2025-01-11 14:09:04 - matgraphdb.materials.core - INFO - Reading materials.
All materials:
  atomic_numbers band_gap.unit  band_gap.value  \
0        [12, 8]          None             NaN   
1        [12, 8]            eV             1.2   

                        cartesian_coords   density  density_atomic elements  \
0  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843        0.103481  [Mg, O]   
1  [[0.0, 0.0, 0.0], [2.13, 2.13, 2.13]]  3.462843        0.103481  [Mg, O]   

  formula                         frac_coords  id  ... structure.lattice.beta  \
0  Mg1 O1  [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]   0  ...                   60.0   
1  Mg1 O1  [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]   1  ...                   60.0   

  structure.lattice.c  structure.lattice.gamma  \
0            3.012275                     60.0   
1            3.012275                     60.0   

                            structure.lattice.matrix structure.lattice.pbc  \
0  [0.0, 2.13, 2.13, 2.13, 0.0, 2.13, 2.13, 2.1

#### Reading Specific Columns

We can also read specific columns from the materials node store. This will only read these columns into memory.

In [18]:
# Read specific columns
materials_subset = mgdb.read_materials(columns=["material_id", "elements"])
print("\nSubset of materials data:")
print(materials_subset.to_pandas())

2025-01-11 14:17:30 - matgraphdb.materials.core - INFO - Reading materials.

Subset of materials data:
  material_id elements
0        mp-1  [Mg, O]
1        mp-2  [Mg, O]


#### Reading with Filters

We can also read materials with filters. This will return a filtered view of the materials node store. This follows the filter syntax of of pyarrow Expressions. More details can be found in the [pyarrow documentation](https://arrow.apache.org/docs/python/compute.html#filtering-by-expressions).


In [12]:
# Read materials with filters
import pyarrow.compute as pc

materials_filtered = mgdb.read_materials(
    columns=["material_id", "elements", "band_gap.value"],
    filters=[pc.field("band_gap.value") == 1.2],
)
print("\nMaterials with band gap == 1.2:")
print(materials_filtered.to_pandas())

2025-01-11 14:09:53 - matgraphdb.materials.core - INFO - Reading materials.

Materials with band gap > 1.0:
  material_id elements  band_gap.value
0        mp-2  [Mg, O]             1.2


#### Filering by ids

You can also filter by ids. This will return the materials with the specified ids. This is just adding a filter on the `id` column. `pc.field("id").isin(ids)`.

In [25]:
materials_filtered = mgdb.read_materials(
    columns=["material_id", "elements", "id"], ids=[1]
)
print("\nMaterials with ids 1:")
print(materials_filtered.to_pandas())

2025-01-11 14:24:20 - matgraphdb.materials.core - INFO - Reading materials.

Materials with ids 1:
  material_id elements  id
0        mp-2  [Mg, O]   1


#### Rebuilding the nested structure of the data

You may have noticed when reading the data some fields are given as `band_gap.value` instead of just `band_gap`. This is because the data is stored in a nested structure. ParquetDB stores the data in a flat structure, so when you read the data, the nested structure is not preserved. You may want to rebuild the nested structure of the data by using the `rebuild_nested_structure` method. This has a cost for the intial read, but subsequent reads will be faster. 

> Note the nested structures are only built the snapshot of the data you read. If you modifiy the data, you will need to rebuild from scratch.


In [24]:
# If you rebuild you can pass the parent column name to the `columns` argument.
materials_rebuilt = mgdb.read_materials(
    columns=["material_id", "elements", "band_gap"],
    rebuild_nested_struct=True,
)
print("\nMaterials:")
print(materials_rebuilt.to_pandas())


# Building from scratch
materials_rebuilt = mgdb.read_materials(
    columns=["material_id", "elements", "band_gap"],
    rebuild_nested_struct=True,
    rebuild_nested_from_scratch=True,
)
print("\nMaterials:")
print(materials_rebuilt.to_pandas())

2025-01-11 14:23:34 - matgraphdb.materials.core - INFO - Reading materials.

Materials:
  material_id elements                       band_gap
0        mp-1  [Mg, O]  {'unit': None, 'value': None}
1        mp-2  [Mg, O]   {'unit': 'eV', 'value': 1.2}
2025-01-11 14:23:34 - matgraphdb.materials.core - INFO - Reading materials.

Materials:
  material_id elements                       band_gap
0        mp-1  [Mg, O]  {'unit': None, 'value': None}
1        mp-2  [Mg, O]   {'unit': 'eV', 'value': 1.2}


#### Loading data in batches

You can also load data in batches. This is useful for loading large amounts of data into memory.

In [15]:
materials_generator = mgdb.read_materials(
    columns=["material_id", "elements", "id"], load_format="batches", batch_size=1
)
print("\nMaterials in batches:")
for i, batch_table in enumerate(materials_generator):
    print(f"Batch {i}:")
    print(batch_table.to_pandas())

2025-01-11 14:15:23 - matgraphdb.materials.core - INFO - Reading materials.

Materials in batches:
Batch 0:
  material_id elements  id
0        mp-1  [Mg, O]   0
Batch 1:
  material_id elements  id
0        mp-2  [Mg, O]   1


#### Accessing the materials node store

In the previous examples, we have used utility method of the `MatGraphDB` class to read the materials. However, you can also access the materials node store directly. The `MaterialNodes` class extends the `NodeStore` which extends the `ParquetDB` class, so you can use all the methods available in `ParquetDB` and `NodeStore`.

In [27]:
materials_store = mgdb.node_stores["materials"]
materials_store = mgdb.material_nodes
materials_store = mgdb.get_node_store(node_type="materials")
print(type(materials_store))

# This method is belongs the the MaterialNodes class, which extends the NodeStore class.
materials = materials_store.read_materials()
print(materials.shape)

# This method is belongs the the NodeStore class.
materials = materials_store.read_nodes()
print(materials.shape)


# This method is belongs the the ParquetDB class.
materials = materials_store.read()
print(materials.shape)

<class 'matgraphdb.materials.nodes.materials.MaterialNodes'>
(2, 32)
(2, 32)
(2, 32)


## 4. Updating Materials

We can update the materials in the database by calling the `update_materials` method. This method will update the materials with the specified `id`. 


In [35]:
# You can update existing materials in the database:
# Update the band gap of of material with id 0

# Read the updated material
updated_material = mgdb.read_materials(
    columns=["id", "material_id", "band_gap.value"],
)

print("Updated material:")
print(updated_material.to_pandas())

update_data = [
    {
        "id": 0,
        "band_gap": {"value": 3.6, "unit": "eV"},
    },
]

mgdb.update_materials(update_data)

# Read the updated material
updated_material = mgdb.read_materials(
    columns=["id", "material_id", "band_gap.value"],
)

print("Updated material:")
print(updated_material.to_pandas())

2025-01-11 14:41:14 - matgraphdb.materials.core - INFO - Reading materials.
Updated material:
   id material_id  band_gap.value
0   0        mp-1             NaN
1   1        mp-2             1.2
2025-01-11 14:41:14 - matgraphdb.materials.core - INFO - Updating materials.
2025-01-11 14:41:14 - matgraphdb.materials.nodes.materials - INFO - Updating data
2025-01-11 14:41:14 - matgraphdb.core.nodes - INFO - Updating 1 node records
2025-01-11 14:41:14 - matgraphdb.materials.nodes.materials - INFO - Data updated successfully.
2025-01-11 14:41:14 - matgraphdb.materials.core - INFO - Reading materials.
Updated material:
   id material_id  band_gap.value
0   0        mp-1             3.6
1   1        mp-2             1.2


#### Updating on a differnt key

You can also update on a different key. This is useful if you want to update the materials with the specified `material_id`. You can also update on multiple keys.



In [36]:
# Read the updated material
updated_material = mgdb.read_materials(
    columns=["id", "material_id", "band_gap.value"],
)

print("Updated material:")
print(updated_material.to_pandas())

update_data = [
    {
        "material_id": "mp-1",
        "band_gap": {"value": 0.1, "unit": "eV"},
    },
]

mgdb.update_materials(update_data, update_keys=["material_id"])


# Read the updated material
updated_material = mgdb.read_materials(
    columns=["id", "material_id", "band_gap.value"],
)

print("Updated material:")
print(updated_material.to_pandas())


update_data = [
    {
        "id": 1,
        "material_id": "mp-1",
        "band_gap": {"value": 0.1, "unit": "eV"},
    },
]

2025-01-11 14:43:00 - matgraphdb.materials.core - INFO - Reading materials.
Updated material:
   id material_id  band_gap.value
0   0        mp-1             3.6
1   1        mp-2             1.2
2025-01-11 14:43:00 - matgraphdb.materials.core - INFO - Updating materials.
2025-01-11 14:43:00 - matgraphdb.materials.nodes.materials - INFO - Updating data
2025-01-11 14:43:00 - matgraphdb.core.nodes - INFO - Updating 1 node records
2025-01-11 14:43:00 - matgraphdb.materials.nodes.materials - INFO - Data updated successfully.
2025-01-11 14:43:00 - matgraphdb.materials.core - INFO - Reading materials.
Updated material:
   id material_id  band_gap.value
0   0        mp-1             0.1
1   1        mp-2             1.2


## 5. Deleting Materials

We can delete the materials in the database by calling the `delete_materials` method. This method will delete the materials with the specified `id`. 

In [37]:
# Delete the material with id 0
mgdb.delete_materials(ids=[0])

# Read the updated material
updated_material = mgdb.read_materials(
    columns=["id", "material_id", "band_gap.value"],
)

print("Updated material:")
print(updated_material.to_pandas())

2025-01-11 14:47:14 - matgraphdb.materials.core - INFO - Deleting materials.
2025-01-11 14:47:14 - matgraphdb.materials.nodes.materials - INFO - Deleting data [0]
2025-01-11 14:47:15 - matgraphdb.materials.nodes.materials - INFO - Data deleted successfully.
2025-01-11 14:47:15 - matgraphdb.materials.core - INFO - Reading materials.
Updated material:
   id material_id  band_gap.value
0   1        mp-2             1.2


#### Deleting columns

You can also delete columns from the materials node store. This will delete the columns from the database.



In [38]:
# Delete the material with id 0
mgdb.delete_materials(columns=["band_gap.value"])

# Read the updated material
updated_material = mgdb.read_materials(
    columns=["id", "material_id", "band_gap.value"],
)

print("Updated material:")
print(updated_material.to_pandas())

2025-01-11 14:48:15 - matgraphdb.materials.core - INFO - Deleting materials.
2025-01-11 14:48:15 - matgraphdb.materials.nodes.materials - INFO - Deleting data None
2025-01-11 14:48:15 - matgraphdb.materials.nodes.materials - INFO - Data deleted successfully.
2025-01-11 14:48:15 - matgraphdb.materials.core - INFO - Reading materials.
Error loading table: No match for FieldRef.Nested(FieldRef.Name(band_gap) FieldRef.Name(value)) in atomic_numbers: list<element: int64>
band_gap.unit: string
cartesian_coords: list<element: list<element: double>>
density: double
density_atomic: double
elements: list<element: string>
formula: string
frac_coords: list<element: list<element: double>>
id: int64
lattice: extension<arrow.fixed_shape_tensor[value_type=double, shape=[3,3]]>
material_id: string
nelements: int64
nsites: int64
source: string
species: list<element: string>
structure.@class: string
structure.@module: string
structure.charge: int64
structure.lattice.a: double
structure.lattice.alpha: dou

# Conclusion

This notebook demonstrated the basic usage of MatGraphDB, including creating, reading, updating, and deleting materials. In the next notebook, we will explore a more complex example and see how we can add and query nodes and edges to the database.